# Exemplo Análise SNA - Aula 8

Neste exemplo iremos apresentar a construção da rede social e apresentar a metodologia de algumas métricas.

## Carregar pacotes

In [ ]:
library(tidyverse)
library(magrittr)
library(lubridate)
library(igraph)
library(visNetwork)

## Carregar dados

In [ ]:
matriz_adjacencia <- read_csv2('/home/vm-data-science/dados/exemplo_sna_aula_8.csv')

In [ ]:
matriz_adjacencia

- vamos realizar um pequeno ajuste na matriz

In [ ]:
m_adj_lower <- matriz_adjacencia %>% 
  select( -id ) %>% 
  as.matrix()

m_adj_lower[ lower.tri(m_adj_lower) ] <- 0

In [ ]:
m_adj_lower

## Análises

### Rede não direcionada

Iremos apresentar a construção e as métricas para uma rede não direcionada.

- *nodes*

In [ ]:
nodes <- matriz_adjacencia %>% 
    distinct(id)
nodes

In [ ]:
edges <- m_adj_lower %>% 
  dplyr::as_data_frame() %>% 
  mutate( from = colnames(m_adj_lower) ) %>% 
  gather( key = to, value = value, -from ) %>% 
  filter( value > 0 )
edges

- Cria rede não direcionada

Nesta rede não há setas direcionando para qual caminho o fluxo está ocorrendo.

In [ ]:
visNetwork(nodes, 
           edges, 
           width = "100%", 
           height = "850",
           main = "Rede Exemplo" ) %>% 
  visInteraction( multiselect = TRUE) %>%
  visOptions( highlightNearest = list(enabled = TRUE, degree = 1),
              nodesIdSelection = TRUE ) %>% 
  visLayout(randomSeed = 541)

- Métricas

In [ ]:
exemplo_network_nao_direct <- graph_from_data_frame( d = edges,
                                                vertices = nodes, 
                                                directed = F )

as.matrix( exemplo_network_nao_direct[] )

- *Density*

In [ ]:
print( edge_density(exemplo_network_nao_direct, loops=F) ) # usando a funcao

print( (mean(degree(exemplo_network_nao_direct))/2) / ((vcount(exemplo_network_nao_direct)-1)/2) ) # calculo na 'mao' 1

print( (sum(degree(exemplo_network_nao_direct))/2) / ((6*(6-1))/2) ) # calculo nao 'mao' 2

- *Degree*

Demonstra se o nó possui muitas conexões com os demais da rede.

In [ ]:
degree(exemplo_network_nao_direct)

- *Closeness*

Mede a capacidade de atuação do nó na rede.

In [ ]:
closeness( exemplo_network_nao_direct, 
           mode = "all", 
           weights = NA, 
           normalized = T )

In [ ]:
# Exemplo do calculo para o no 2
1 / ( sum( distances( exemplo_network_nao_direct )[2,] ) / ( vcount(exemplo_network_nao_direct)-1) )

- *Betweenness*

O quanto o nó influencia no fluxo da informação na rede, a capacidade de intermediação.

In [ ]:
betweenness(exemplo_network_nao_direct, 
            directed = F, 
            weights = NA, 
            normalized = F)

O cálculo do *betweenness* merece um pouco mais de detalhes para entendimento. A medida verifica se o determinado nó está no meio do caminho da menor distância entre dois nós.

Vamos mostrar detalhes para o nó 3.

- caminho(s) mais curto(s) entre os nós 1 e 2:

In [ ]:
all_shortest_paths( exemplo_network_nao_direct, from = 1, to = 2 )

Temos 1 menor caminho, mas o nó 3 não pertence, então 0/1

- caminho(s) mais curto(s) entre os nós 1 e 4:

In [ ]:
all_shortest_paths( exemplo_network_nao_direct, from = 1, to = 4 )

Temos 1 menor caminho, mas o nó 3 não pertence, então 0/1

- caminho(s) mais curto(s) entre os nós 1 e 5:

In [ ]:
all_shortest_paths( exemplo_network_nao_direct, from = 1, to = 5 )

Temos 1 menor caminho, mas o nó 3 não pertence, então 0/1

- caminho(s) mais curto(s) entre os nós 1 e 6:

In [ ]:
all_shortest_paths( exemplo_network_nao_direct, from = 1, to = 6 )

Temos 2 menores caminhos e o nó 3 pertence aos 2, então 2/2

- caminho(s) mais curto(s) entre os nós 2 e 4:

In [ ]:
all_shortest_paths( exemplo_network_nao_direct, from = 2, to = 4 )

Temos 1 menor caminho, mas o nó 3 não pertence, então 0/1

- caminho(s) mais curto(s) entre os nós 2 e 5:

In [ ]:
all_shortest_paths( exemplo_network_nao_direct, from = 2, to = 5 )

Temos 2 menores caminhos, mas o nó 3 somente pertence a 1 deles, então 1/2.

Podemos verificar que o nó 3 está contido em todos os demais menores caminhos.

In [ ]:
all_shortest_paths( exemplo_network_nao_direct, from = 2, to = 6 ) # 1/1
all_shortest_paths( exemplo_network_nao_direct, from = 4, to = 5 ) # 1/1
all_shortest_paths( exemplo_network_nao_direct, from = 4, to = 6 ) # 1/1
all_shortest_paths( exemplo_network_nao_direct, from = 5, to = 6 ) # 1/1

Ao somarmos todos os valores temos o *betweenness*

### Rede direcionada

A rede direcionada mostra o fluxo de como a informação pode seguir.

In [ ]:
visNetwork(nodes, 
           edges, 
           width = "100%", 
           height = "850",
           main = "Rede Exemplo" ) %>% 
  visEdges(arrows = 'to') %>%
  visInteraction( multiselect = TRUE) %>%
  visOptions( highlightNearest = list(enabled = TRUE, degree = 1),
              nodesIdSelection = TRUE ) %>% 
  visLayout(randomSeed = 541)

- Métricas

In [ ]:
exemplo_network_direct <- graph_from_data_frame( d = edges,
                                                vertices = nodes, 
                                                directed = T )

as.matrix( exemplo_network_direct[] )

- *Density*

In [ ]:
print( edge_density(exemplo_network_direct, loops=F) ) # usando a funcao

print( (mean(degree(exemplo_network_direct, mode = 'out'))/2) / ((vcount(exemplo_network_direct)-1)/2) ) # calculo na 'mao' 1

print( (sum(degree(exemplo_network_direct, mode = 'out'))/2) / ((6*(6-1))/2) )

- *Closeness*

In [ ]:
closeness( exemplo_network_direct, 
           mode = "out", 
           weights = NA, 
           normalized = T )

- *Betweenness*

In [ ]:
betweenness(exemplo_network_direct, 
            directed = T, 
            weights = NA, 
            normalized = F)